In [2]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [27]:
from langchain_together import ChatTogether, TogetherEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [ ]:

llm = ChatTogether(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0
)

In [6]:
system_message = SystemMessage(
    content="You are Autobot, an IITM BS degree virtual assistant for the students of the BS Data Science program. You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know."
    )

In [61]:
prompt = ChatPromptTemplate.from_messages([
    system_message,  # System prompt
    MessagesPlaceholder(variable_name="retrieved_context"),
    ("human", "{user_query}") 
])

In [47]:
prompt

ChatPromptTemplate(input_variables=['retrieved_context', 'user_query'], input_types={'retrieved_context': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typ

In [12]:
autobot = prompt | llm

In [29]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
loader = TextLoader("./assets/Intro-to-python.md")
documents = loader.load()



In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 720 , chunk_overlap = 72
)
chunks = text_splitter.split_documents(documents)

In [26]:
chunks[0].page_content

'# Intro to Python Programming - Course Information\n\n## Grading Information\n\n### Quiz Dates:\n- **Quiz 1:** February 23, 2025\n- **Quiz 2:** No Quiz 2\n- **End Term Exam:** April 13, 2025\n\n### Programming Exams (OPPE):\n- **OPPE1:** Sunday, March 2, 2025\n- **OPPE2:** Sunday, April 6, 2025\n\n### OPPE Slot Allocation:\nDepending on eligibility for OPPE1 & OPPE2, students will be allocated one of three slots. Ensure availability on the given dates.\n\n### Eligibility for Bonus:\n- The **SCT (System Compatibility Test)** is mandatory for the bonus to be added to the final score.\n- Attendance in mock tests alone does **not** qualify for the bonus.\n\n## Eligibility Criteria'

In [28]:
embedding_model = TogetherEmbeddings()

In [31]:
vector_store = FAISS.from_documents(chunks , embedding_model)

In [59]:

def reterive_context(user_query):
    
    docs = vector_store.similarity_search(user_query, k=3)
    content =  "\n".join([doc.page_content for doc in docs])
    return [SystemMessage(content=content)]


In [60]:
print(reterive_context("Can you tell me about the grading of the python course"))

[SystemMessage(content='# Intro to Python Programming - Course Information\n\n## Grading Information\n\n### Quiz Dates:\n- **Quiz 1:** February 23, 2025\n- **Quiz 2:** No Quiz 2\n- **End Term Exam:** April 13, 2025\n\n### Programming Exams (OPPE):\n- **OPPE1:** Sunday, March 2, 2025\n- **OPPE2:** Sunday, April 6, 2025\n\n### OPPE Slot Allocation:\nDepending on eligibility for OPPE1 & OPPE2, students will be allocated one of three slots. Ensure availability on the given dates.\n\n### Eligibility for Bonus:\n- The **SCT (System Compatibility Test)** is mandatory for the bonus to be added to the final score.\n- Attendance in mock tests alone does **not** qualify for the bonus.\n\n## Eligibility Criteria\n## Final Course Score Calculation:\nFormula: **Capped at 100**.\n\nWhere:\n- **GAA1:** Average score in the **best 10 out of 11** graded objective assignments.\n- **GAA2:** Average score in the **best 10 out of 11** graded programming assignments.\n- **Qz1:** Quiz 1 score (**0** if not at

In [62]:
def call_autobot(user_query,reterived_context):
   return autobot.invoke({"user_query":user_query,"retrieved_context":reterived_context})
    

In [65]:
user_query="Tell me about the grading subject of the python course"
reterived_context = reterive_context(user_query)
response = call_autobot(user_query,reterived_context)
print(response.content)

The grading for the **Intro to Python Programming** course is based on the following components:

1. **Graded Objective Assignments (GAA1):**  
   - The average score of the **best 10 out of 11** graded objective assignments.

2. **Graded Programming Assignments (GAA2):**  
   - The average score of the **best 10 out of 11** graded programming assignments.

3. **Quiz 1 (Qz1):**  
   - The score of Quiz 1 (if not attempted, the score is **0**).

4. **Programming Exams (OPPE):**  
   - **OPPE1 (PE1):** Score of the first programming exam (if not attempted, the score is **0**).  
   - **OPPE2 (PE2):** Score of the second programming exam (if not attempted, the score is **0**).

5. **Final Exam (F):**  
   - The score of the end-term exam.

### Final Course Score Calculation:
The final score is calculated using the formula:  
**Final Score = min(100, 0.15 × GAA1 + 0.15 × GAA2 + 0.10 × Qz1 + 0.20 × PE1 + 0.20 × PE2 + 0.20 × F)**

### Additional Notes:
- The final score is **capped at 100**.